In [10]:
import pickle
import pandas as pd
import numpy as np
import unicodedata

In [5]:
with open("logradouros.pkl", "rb") as fp:
    ADDRESSES = pickle.load(fp)

In [8]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [6]:
estabelecimentos = pd.read_csv("/home/bossa/Projetos/MapaSaudeSC/data/estabelecimentos-420000-202202.csv")

In [19]:
np.argmax([1,2,5,3,2])

2

In [20]:
FILTRADO = {}

In [22]:

for cnes, registry in ADDRESSES.items():
    [bairro, cidade, cep] = estabelecimentos.query(f"CNES == {cnes}")[["BAIRRO","MUNICIPIO","CEP"]].values[0] 
    if len(registry) == 0:
        pass
    elif len(registry) == 1:
        FILTRADO[cnes] = registry[0]
    else:
        BIGCHECK = []
        for x in registry:
            displayname = remove_accents(x['display_name']).lower()
            cep = str(cep)
            cep = cep[:5] + "-" +  cep[-3:]
            chekings = [bairro.lower() in displayname, cidade.lower() in displayname, cep in displayname, cep[0:5] in displayname, cep[0:8] in displayname]
            BIGCHECK.append(sum(chekings))
        ii = np.argmax(BIGCHECK)
        FILTRADO[cnes] = registry[ii]

In [24]:
LATITUDES = {key: float(val['lat']) for (key, val) in FILTRADO.items()}
LONGITUDES = {key: float(val['lon']) for (key, val) in FILTRADO.items()}
geocodes = pd.DataFrame({"lat": LATITUDES, "lon": LONGITUDES})